<a href="https://colab.research.google.com/github/Rabinmuhd/Assignments/blob/main/Data_Acquisition_Casestudy_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sqlite3

# Step 1: Load SpaceX Launch Data from API
 ● Use https://api.spacexdata.com/v4/launches

In [9]:
import requests
url = "https://api.spacexdata.com/v4/launches"
response=requests.get(url)
data=response.json()
launch_df=pd.DataFrame(data)
launch_df

,fairings,links,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,...,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id
0,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,...,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a
1,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,...,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight'...",True,False,None,5eb87cdaffd86e000604b32b
2,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'resi...",Residual stage 1 thrust led to collision betwe...,...,2008-08-03T03:34:00.000Z,1217734440,2008-08-03T15:34:00+12:00,hour,False,"[{'core': '5e9e289ef3591814873b2625', 'flight'...",True,False,None,5eb87cdbffd86e000604b32c
3,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],Ratsat was carried to orbit on the first succe...,...,2008-09-28T23:15:00.000Z,1222643700,2008-09-28T11:15:00+12:00,hour,False,"[{'core': '5e9e289ef3591855dc3b2626', 'flight'...",True,False,None,5eb87cdbffd86e000604b32d
4,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,True,[],None,...,2009-07-13T03:35:00.000Z,1247456100,2009-07-13T15:35:00+12:00,hour,False,"[{'core': '5e9e289ef359184f103b2627', 'flight'...",True,False,None,5eb87cdcffd86e000604b32e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,"{'reused': None, 'recovery_attempt': None, 're...","{'patch': {'small': None, 'large': None}, 'red...",None,NaN,False,NaN,5e9d0d95eda69973a809d1ec,None,[],None,...,2022-12-01T00:00:00.000Z,1669852800,2022-11-30T19:00:00-05:00,month,True,"[{'core': None, 'flight': None, 'gridfins': Tr...",True,False,None,633f72580531f07b4fdf59c6
201,"{'reused': None, 'recovery_attempt': None, 're...","{'patch': {'small': None, 'large': None}, 'red...",None,NaN,False,NaN,5e9d0d95eda69973a809d1ec,None,[],None,...,2022-12-01T00:00:00.000Z,1669852800,2022-11-30T16:00:00-08:00,month,True,"[{'core': None, 'flight': None, 'gridfins': No...",True,False,None,63161384ffc78f3b8567070e
202,"{'reused': None, 'recovery_attempt': None, 're...","{'patch': {'small': None, 'large': None}, 'red...",None,NaN,False,NaN,5e9d0d95eda69973a809d1ec,None,[],None,...,2022-12-01T00:00:00.000Z,1669852800,2022-11-30T19:00:00-05:00,month,True,"[{'core': None, 'flight': None, 'gridfins': No...",True,False,None,6243ae58af52800c6e91925a
203,"{'reused': None, 'recovery_attempt': None, 're...","{'patch': {'small': None, 'large': None}, 'red...",None,NaN,False,NaN,5e9d0d95eda69974db09d1ed,None,[],None,...,2022-12-01T00:00:00.000Z,1669852800,2022-11-30T19:00:00-05:00,month,True,"[{'core': None, 'flight': None, 'gridfins': Tr...",True,False,None,633f72130531f07b4fdf59c3


● Extract relevant columns: name, date_utc, success, details, rocket

In [11]:
launch_new = launch_df[['name','date_utc','success','details','rocket']]

● Convert date_utc to datetime and extract the year


In [12]:
launch_df['date_utc'] = pd.to_datetime(launch_df['date_utc'])
launch_df['year'] = launch_df['date_utc'].dt.year
launch_df.head()

,fairings,links,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,...,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,year
0,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,...,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a,2006
1,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,...,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight'...",True,False,None,5eb87cdaffd86e000604b32b,2007
2,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'resi...",Residual stage 1 thrust led to collision betwe...,...,1217734440,2008-08-03T15:34:00+12:00,hour,False,"[{'core': '5e9e289ef3591814873b2625', 'flight'...",True,False,None,5eb87cdbffd86e000604b32c,2008
3,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],Ratsat was carried to orbit on the first succe...,...,1222643700,2008-09-28T11:15:00+12:00,hour,False,"[{'core': '5e9e289ef3591855dc3b2626', 'flight'...",True,False,None,5eb87cdbffd86e000604b32d,2008
4,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,True,[],None,...,1247456100,2009-07-13T15:35:00+12:00,hour,False,"[{'core': '5e9e289ef359184f103b2627', 'flight'...",True,False,None,5eb87cdcffd86e000604b32e,2009


# Step 2: Load Rocket Metadata
● Use https://api.spacexdata.com/v4/rockets

In [13]:
url1='https://api.spacexdata.com/v4/rockets'
response=requests.get(url1)
data2=response.json()
rockets_df=pd.DataFrame(data2)
rockets_df

,height,diameter,mass,first_stage,second_stage,engines,landing_legs,payload_weights,flickr_images,name,...,stages,boosters,cost_per_launch,success_rate_pct,first_flight,country,company,wikipedia,description,id
0,"{'meters': 22.25, 'feet': 73}","{'meters': 1.68, 'feet': 5.5}","{'kg': 30146, 'lb': 66460}","{'thrust_sea_level': {'kN': 420, 'lbf': 94000}...","{'thrust': {'kN': 31, 'lbf': 7000}, 'payloads'...","{'isp': {'sea_level': 267, 'vacuum': 304}, 'th...","{'number': 0, 'material': None}","[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...","[https://imgur.com/DaCfMsj.jpg, https://imgur....",Falcon 1,...,2,0,6700000,40,2006-03-24,Republic of the Marshall Islands,SpaceX,https://en.wikipedia.org/wiki/Falcon_1,The Falcon 1 was an expendable launch system p...,5e9d0d95eda69955f709d1eb
1,"{'meters': 70, 'feet': 229.6}","{'meters': 3.7, 'feet': 12}","{'kg': 549054, 'lb': 1207920}","{'thrust_sea_level': {'kN': 7607, 'lbf': 17100...","{'thrust': {'kN': 934, 'lbf': 210000}, 'payloa...","{'isp': {'sea_level': 288, 'vacuum': 312}, 'th...","{'number': 4, 'material': 'carbon fiber'}","[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...",[https://farm1.staticflickr.com/929/2878733830...,Falcon 9,...,2,0,50000000,98,2010-06-04,United States,SpaceX,https://en.wikipedia.org/wiki/Falcon_9,Falcon 9 is a two-stage rocket designed and ma...,5e9d0d95eda69973a809d1ec
2,"{'meters': 70, 'feet': 229.6}","{'meters': 12.2, 'feet': 39.9}","{'kg': 1420788, 'lb': 3125735}","{'thrust_sea_level': {'kN': 22819, 'lbf': 5130...","{'thrust': {'kN': 934, 'lbf': 210000}, 'payloa...","{'isp': {'sea_level': 288, 'vacuum': 312}, 'th...","{'number': 12, 'material': 'carbon fiber'}","[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...",[https://farm5.staticflickr.com/4599/385838292...,Falcon Heavy,...,2,2,90000000,100,2018-02-06,United States,SpaceX,https://en.wikipedia.org/wiki/Falcon_Heavy,With the ability to lift into orbit over 54 me...,5e9d0d95eda69974db09d1ed
3,"{'meters': 118, 'feet': 387}","{'meters': 9, 'feet': 30}","{'kg': 1335000, 'lb': 2943000}","{'thrust_sea_level': {'kN': 128000, 'lbf': 287...","{'thrust': {'kN': 1957, 'lbf': 440000}, 'paylo...","{'isp': {'sea_level': 330, 'vacuum': 380}, 'th...","{'number': 6, 'material': 'stainless steel'}","[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...",[https://live.staticflickr.com/65535/489541389...,Starship,...,2,0,7000000,0,2021-12-01,United States,SpaceX,https://en.wikipedia.org/wiki/SpaceX_Starship,Starship and Super Heavy Rocket represent a fu...,5e9d0d96eda699382d09d1ee


● Extract id, name, type, active, and stages


In [18]:
rockets_new = rockets_df[['id','type','active','stages']]
rockets_new

,id,type,active,stages
0,5e9d0d95eda69955f709d1eb,rocket,False,2
1,5e9d0d95eda69973a809d1ec,rocket,True,2
2,5e9d0d95eda69974db09d1ed,rocket,True,2
3,5e9d0d96eda699382d09d1ee,rocket,False,2


# Step 3: Merge Launch and Rocket Data
● Join the two DataFrames on rocket ID using


In [17]:
merged_df = pd.merge(launch_new,rockets_new,left_on='rocket', right_on='id', how='left')
merged_df

,name,date_utc,success,details,rocket,id,type,active,stages
0,FalconSat,2006-03-24T22:30:00.000Z,False,Engine failure at 33 seconds and loss of vehicle,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,rocket,False,2
1,DemoSat,2007-03-21T01:10:00.000Z,False,Successful first stage burn and transition to ...,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,rocket,False,2
2,Trailblazer,2008-08-03T03:34:00.000Z,False,Residual stage 1 thrust led to collision betwe...,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,rocket,False,2
3,RatSat,2008-09-28T23:15:00.000Z,True,Ratsat was carried to orbit on the first succe...,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,rocket,False,2
4,RazakSat,2009-07-13T03:35:00.000Z,True,None,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,rocket,False,2
...,...,...,...,...,...,...,...,...,...
200,Transporter-6,2022-12-01T00:00:00.000Z,None,None,5e9d0d95eda69973a809d1ec,5e9d0d95eda69973a809d1ec,rocket,True,2
201,TTL-1,2022-12-01T00:00:00.000Z,None,None,5e9d0d95eda69973a809d1ec,5e9d0d95eda69973a809d1ec,rocket,True,2
202,WorldView Legion 1 & 2,2022-12-01T00:00:00.000Z,None,None,5e9d0d95eda69973a809d1ec,5e9d0d95eda69973a809d1ec,rocket,True,2
203,Viasat-3 & Arcturus,2022-12-01T00:00:00.000Z,None,None,5e9d0d95eda69974db09d1ed,5e9d0d95eda69974db09d1ed,rocket,True,2


# Step 4: Add Simulated Country Information
● Add a new column country and randomly assign one of these values:
['USA', 'Russia', 'India', 'China', 'France']


In [19]:
countries = ['USA', 'Russia', 'India', 'China', 'France']
merged_df['country_simulated'] = np.random.choice(countries, size=len(merged_df))
merged_df.head()

,name,date_utc,success,details,rocket,id,type,active,stages,country_simulated
0,FalconSat,2006-03-24T22:30:00.000Z,False,Engine failure at 33 seconds and loss of vehicle,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,rocket,False,2,USA
1,DemoSat,2007-03-21T01:10:00.000Z,False,Successful first stage burn and transition to ...,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,rocket,False,2,France
2,Trailblazer,2008-08-03T03:34:00.000Z,False,Residual stage 1 thrust led to collision betwe...,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,rocket,False,2,France
3,RatSat,2008-09-28T23:15:00.000Z,True,Ratsat was carried to orbit on the first succe...,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,rocket,False,2,China
4,RazakSat,2009-07-13T03:35:00.000Z,True,None,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,rocket,False,2,USA


# Step 5: Store Merged Data in SQLite3
● Use sqlite3 to create a connection and save the merged DataFrame as a table named
launches
● Table should contain all merged columns including country

In [20]:
conn = sqlite3.connect('spacex.db')
merged_df.to_sql('launches', conn, if_exists='replace', index=False)

205

# Step 6: Run SQL Queries on the Data to analyze
1. Launches by Country

In [22]:
query = """SELECT country_simulated AS country, COUNT(*) AS launch_count
FROM launchesGROUP BY country_simulated ORDER BY launch_count DESC;"""

pd.read_sql_query(query, conn)

,country,launch_count
0,China,48
1,India,47
2,France,43
3,USA,38
4,Russia,29


2. Which year had the highest number of launches?


In [23]:
query = """
SELECT strftime('%Y', date_utc) AS year, COUNT(*) AS launch_count
FROM launches GROUP BY year ORDER BY launch_count DESC LIMIT 1;"""

pd.read_sql_query(query, conn)

,year,launch_count
0,2022,62


3. Top 5 Missions by Launch Count

In [25]:
query = """
SELECT name AS mission_name, COUNT(*) AS launch_count FROM launches
GROUP BY mission_name ORDER BY launch_count DESCLIMIT 5;"""

pd.read_sql_query(query, conn)

,mission_name,launch_count
0,ispace Mission 1 & Rashid,1
1,ZUMA,1
2,WorldView Legion 1 & 2,1
3,Viasat-3 & Arcturus,1
4,USSF-44,1
